In [4]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

In [5]:
right = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\NDVI\DIM_PNEO3_202301200802330_PMS-FS_ORT_PWOI_000160930_9_1_F_1-NDVI.tif"
left = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO4_202302280802132_PMS-FS_ORT_PWOI_000160930_14_1_F_1-NDVI-BIN.tif"
poly=r"D:\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\08-ndvi\intersect.gpkg"

In [6]:
lsrc=    rasterio.open(left)
rsrc=    rasterio.open(right)  
polygpd = gpd.read_file(poly)
lout, _ = mask(lsrc, polygpd.geometry, invert=False)
rout, _ = mask(rsrc, polygpd.geometry, invert=False)


r_data = rsrc.read(1) 
l_data = lsrc.read(1)

r_data[r_data==-9] = np.nan
l_data[l_data==-9] = np.nan

ValueError: cannot convert float NaN to integer

In [7]:
l_dataf=l_data.flatten()
r_dataf=r_data.flatten()

In [13]:
from sklearn import datasets, linear_model
reg = linear_model.LinearRegression()

reg.fit(X=l_dataf.reshape(-1, 1),y=r_dataf.reshape(-1, 1))

ValueError: Input y contains NaN.

In [ ]:
def process_image(src_img, dst_img, band_id=1):
    with rasterio.open(src_img) as src:

        tiles = src.block_windows(band_id)
        profile = src.profile
        p_count = src.shape[0]*src.shape[1]
        # ndval = src.nodatavals[0]
        # print(ndval)
        profile.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw',
            BIGTIFF = "IF_SAFER")
        with rasterio.open(dst_img, 'w', **profile) as dst:
            for idx, window in tiles:
                # print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
                
                # dst_data = src.read(band_id, window=window) #read all 7 bands
                # read bands
                band1 = src.read(1, window=window)
                band2 = src.read(2, window=window)
                band3 = src.read(3, window=window)
                band4 = src.read(4, window=window)
                band5 = src.read(5, window=window)
                band6 = src.read(6, window=window)
                band7 = src.read(7, window=window)
                band8 = src.read(8, window=window)
                print(((((idx[0]+1)*band1.shape[0]*band1.shape[1])/p_count)*100))
                # convert numpy
                npbands = np.array([band1, band2, band3, band4, band5, band6, band7, band8])
                np.nan_to_num(npbands, copy=False)
                npbands = pd.DataFrame(npbands.reshape([8,-1]).T)
                # print(npbands.head)

                

                npbands.columns = ['blue', 'green', 'red', 'nir', 'arvi', 'EVI', 'NDVI','SAVI']
                        
                # do predict
                out = model.predict(npbands)
                
                # reconvert to raster with georef
                # pred_raster_int = np.unique(out, return_inverse = True)[1]
                
                height, width = band1.shape
                pred_raster = np.array(out).reshape((height, width))
                
                # write with rasterio
                dst.write_band(band_id, pred_raster, window=window)
    return